In [3]:
# 依旧先导包
# 老规矩，先导包和中文解码问题
import numpy as np
import pandas as pd
import seaborn as sns

# 解决中文显示问题，下面的代码只需运行一次即可
import matplotlib as plt

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
import os
# 查看当前文件夹下的所有文件
list_dir = os.listdir()
list_dir

['.git',
 '.idea',
 'analyse_dataset.ipynb',
 'dataset',
 'model',
 'models.py',
 'README.md',
 'requirements.txt',
 'train.py',
 'utils.py',
 '__pycache__',
 '研究日记.md']

In [6]:
# 色散的网络数据集
disper_comsol_df = pd.read_excel('./dataset/色散comsol+网络.xlsx')
disper_comsol_df = disper_comsol_df.iloc[:8793]
# 有效折射率差值的网络数据集
diff_neff_comsol_df = pd.read_excel('./dataset/Δneff_comsol+网络.xlsx')
diff_neff_comsol_df = diff_neff_comsol_df.iloc[:11545]

In [7]:
disper_comsol_df

,n1,n2,r8,r9,r10,wl,HE11,HE21,TE01,TM01,HE31,EH11,HE41,EH21,HE51,EH31,HE61,EH41,8793(含)前为comsol
0,0.047,-0.010,3.00,6.00,7.50,1.52,33.404962,36.836901,35.032116,39.504573,46.620367,47.211598,61.746321,61.243792,80.006561,77.771623,97.024585,92.460164,NaN
1,0.047,-0.010,3.00,6.00,7.50,1.54,34.275525,37.715470,35.951637,40.327533,47.531142,48.086259,62.635635,62.067859,80.634301,78.281667,96.609501,91.942180,NaN
2,0.047,-0.010,3.00,6.00,7.50,1.56,35.107158,38.553291,36.833222,41.105127,48.395522,48.911955,63.462941,62.826827,81.154780,78.681392,95.918255,91.149635,NaN
3,0.047,-0.010,3.00,6.00,7.50,1.58,35.902281,39.352737,37.679280,41.839651,49.215627,49.690757,64.229378,63.521782,81.564871,78.967812,94.918684,90.031097,NaN
4,0.047,-0.010,4.30,6.70,7.70,1.52,23.150625,25.074144,24.109711,25.954057,30.958200,30.682171,40.391599,39.612137,52.586476,50.919311,65.102172,62.506007,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8788,0.075,-0.045,3.40,5.70,7.20,1.56,48.160892,53.653948,51.040626,62.651451,65.949348,70.292201,85.996680,88.054372,113.738602,113.105554,148.665343,144.148758,NaN
8789,0.075,-0.045,3.40,5.70,7.00,1.54,47.515412,52.419054,48.156965,61.355684,64.515774,68.846439,84.248836,86.329079,111.495476,110.950347,145.705303,141.454020,NaN
8790,0.075,-0.045,3.40,5.70,7.00,1.55,47.811424,52.755919,49.221971,61.629788,64.904964,69.176386,84.728564,86.744205,112.087594,111.439988,146.390287,141.972183,NaN
8791,0.075,-0.045,3.40,5.70,7.00,1.56,48.030824,53.080862,51.829430,61.888679,65.281304,69.490996,85.193784,87.142380,112.661666,111.908612,147.048959,142.460490,NaN


In [8]:
diff_neff_comsol_df

,n1,n2,r8,r9,r10,wl,TE01-HE21,HE21-TM01,HE31-EH11,HE41-EH21,HE51-EH31,HE61-EH41,11545(含)前为comsol
0,0.047,-0.010,3.00,6.00,7.50,1.50,4.797212,5.885985,1.184874,0.491146,0.293180,1.384228,NaN
1,0.047,-0.010,3.00,6.00,7.50,1.52,4.944192,6.075697,1.228810,0.503493,0.319668,1.465229,NaN
2,0.047,-0.010,3.00,6.00,7.50,1.54,5.092597,6.267515,1.273213,0.515443,0.347921,1.549832,NaN
3,0.047,-0.010,3.00,6.00,7.50,1.56,5.242376,6.461369,1.318049,0.526950,0.378007,1.638073,NaN
4,0.047,-0.010,3.00,6.00,7.50,1.58,5.393479,6.657186,1.363281,0.537969,0.409995,1.729982,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11540,0.050,-0.030,5.05,7.00,7.92,1.56,10.700164,20.974487,11.789200,7.639810,5.150028,3.352579,NaN
11541,0.060,-0.040,4.92,7.03,8.01,1.54,10.229578,19.412025,10.571555,6.793452,4.582997,2.971351,NaN
11542,0.060,-0.040,4.92,7.03,8.01,1.55,10.382716,19.740784,10.771508,6.926992,4.675942,3.030476,NaN
11543,0.060,-0.040,4.92,7.03,8.01,1.56,10.536349,20.070406,10.971865,7.060732,4.768889,3.089314,NaN


In [9]:
# 色散数据集的最大最小值
disper_comsol_df.describe().loc[['count','min','max']]

,n1,n2,r8,r9,r10,wl,HE11,HE21,TE01,TM01,HE31,EH11,HE41,EH21,HE51,EH31,HE61,EH41,8793(含)前为comsol
count,8793.000,8793.00,8793.000,8793.0,8793.0,8793.00,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,8793.000000,0.0
min,0.018,-0.05,2.709,5.7,7.0,1.51,-47.864918,-49.797800,-47.073422,-115.965649,-51.621914,-115.537541,-53.496668,-114.096360,-56.356086,-115.267863,-62.296721,-115.907453,NaN
max,0.090,-0.01,8.100,14.0,16.0,1.59,51.093308,55.906172,51.829430,62.764967,68.885087,71.384960,89.993278,90.689616,118.444217,117.836674,153.609636,148.524927,NaN


In [40]:
# 先计算两个差值
diff_r3_r2 = disper_comsol_df['r10'] - disper_comsol_df['r9']
diff_r2_r1 = disper_comsol_df['r9'] - disper_comsol_df['r8']

# 合并为新DataFrame（指定列名，更易读）
diff_df = pd.DataFrame({
    'diff_r3_r2': diff_r3_r2,  # 列名1：r10 - r9
    'diff_r2_r1': diff_r2_r1   # 列名2：r9 - r8
})

diff_df.describe()

,diff_r3_r2,diff_r2_r1
count,8793.000000,8793.000000
mean,1.084347,2.482746
std,0.198466,1.067835
min,0.600000,1.300000
25%,1.000000,1.800000
50%,1.000000,2.300000
75%,1.100000,2.950000
max,4.500000,8.800000


In [38]:
diff_neff_comsol_df.describe().loc[['count','min','max']]

,n1,n2,r8,r9,r10,wl,TE01-HE21,HE21-TM01,HE31-EH11,HE41-EH21,HE51-EH31,HE61-EH41,11545(含)前为comsol
count,11545.000,11545.00,11545.000,11545.0,11545.0,11545.0,11545.000000,11545.000000,11545.000000,11545.000000,11545.000000,11545.000000,0.0
min,0.018,-0.05,2.709,5.7,7.0,1.5,0.257276,0.274097,0.015969,0.003171,0.002987,0.016602,NaN
max,0.090,-0.01,8.100,14.0,16.0,1.6,13.220317,66.230488,56.617398,47.784099,41.014492,33.247190,NaN


In [41]:
# 先计算两个差值
diff_r3_r2 = diff_neff_comsol_df['r10'] - diff_neff_comsol_df['r9']
diff_r2_r1 = diff_neff_comsol_df['r9'] - diff_neff_comsol_df['r8']

# 合并为新DataFrame（指定列名，更易读）
diff_df = pd.DataFrame({
    'diff_r3_r2': diff_r3_r2,  # 列名1：r10 - r9
    'diff_r2_r1': diff_r2_r1   # 列名2：r9 - r8
})

diff_df.describe()

,diff_r3_r2,diff_r2_r1
count,11545.000000,11545.000000
mean,1.072984,2.459582
std,0.161427,1.031975
min,0.000000,1.400000
25%,1.000000,1.800000
50%,1.000000,2.300000
75%,1.100000,2.950000
max,2.127000,8.800000


In [47]:
for column in disper_comsol_df.columns:
    val_counts_df = disper_comsol_df[column].value_counts(dropna=False).reset_index()
    val_counts_df.columns = ['元素', '出现次数']
    val_counts_df = val_counts_df.sort_values(by='元素', ascending=True, na_position='last')
    print(f'列{column}的统计：')
    print(val_counts_df.head(20))

列n1的统计：
        元素  出现次数
36  0.0180     9
53  0.0200     3
1   0.0250   726
2   0.0300   702
11  0.0350   481
35  0.0380    12
34  0.0390    12
8   0.0400   567
49  0.0410     3
54  0.0420     3
13  0.0440   309
39  0.0450     6
12  0.0470   386
38  0.0471     6
14  0.0480    78
19  0.0490    36
10  0.0500   488
15  0.0510    63
17  0.0520    48
20  0.0530    33
列n2的统计：
        元素  出现次数
41 -0.0500     3
9  -0.0490   135
42 -0.0485     3
7  -0.0470   620
19 -0.0460    24
5  -0.0450   878
32 -0.0440     9
11 -0.0430    60
20 -0.0420    21
37 -0.0410     3
6  -0.0400   848
34 -0.0390     9
15 -0.0380    36
28 -0.0370    15
27 -0.0360    15
4  -0.0350   913
36 -0.0340     6
18 -0.0330    27
26 -0.0320    15
12 -0.0310    57
列r8的统计：
        元素  出现次数
193  2.709     3
187  2.747     3
191  2.782     3
196  2.829     3
197  2.844     3
195  2.858     3
190  2.928     3
75   2.932     6
185  2.947     3
168  2.950     3
3    3.000   487
198  3.001     3
74   3.004     6
186  3.010     3
165  3.